In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-05-19 10:40:28--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  4.89MB/s    in 0.2s    

2025-05-19 10:40:28 (4.89 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()

In [3]:
print(f"length of text in characters is: {len(text)}")

length of text in characters is: 1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
##craeting the encoders and decoders
stoi = {ch:i for i,ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [7]:
print(encode("s&ck my d&ck"))
print(decode(encode("s&ck my d&ck")))

[57, 4, 41, 49, 1, 51, 63, 1, 42, 4, 41, 49]
s&ck my d&ck


In [8]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
##split into train and test
n = int(0.85*len(data))
train_data = data[:n]
test_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} target is: {target}")

when input is tensor([18]) target is: 47
when input is tensor([18, 47]) target is: 56
when input is tensor([18, 47, 56]) target is: 57
when input is tensor([18, 47, 56, 57]) target is: 58
when input is tensor([18, 47, 56, 57, 58]) target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is: 58


In [12]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split):
  data = train_data if split == 'train' else test_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b,:t+1]
    target = yb[b,t]
    print(f"when input is {context} target is: {target}")

inputs:
torch.Size([4, 8])
tensor([[53, 61, 57, 10,  0, 20, 43,  1],
        [39, 41, 43, 42,  1, 58, 46, 43],
        [52, 41, 43,  8,  0,  0, 24, 17],
        [26, 33, 31, 10,  0, 25, 53, 57]])
targets:
torch.Size([4, 8])
tensor([[61, 57, 10,  0, 20, 43,  1, 58],
        [41, 43, 42,  1, 58, 46, 43,  1],
        [41, 43,  8,  0,  0, 24, 17, 27],
        [33, 31, 10,  0, 25, 53, 57, 58]])
----
when input is tensor([53]) target is: 61
when input is tensor([53, 61]) target is: 57
when input is tensor([53, 61, 57]) target is: 10
when input is tensor([53, 61, 57, 10]) target is: 0
when input is tensor([53, 61, 57, 10,  0]) target is: 20
when input is tensor([53, 61, 57, 10,  0, 20]) target is: 43
when input is tensor([53, 61, 57, 10,  0, 20, 43]) target is: 1
when input is tensor([53, 61, 57, 10,  0, 20, 43,  1]) target is: 58
when input is tensor([39]) target is: 41
when input is tensor([39, 41]) target is: 43
when input is tensor([39, 41, 43]) target is: 42
when input is tensor([39, 41,

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)


  def forward(self, idx, targets = None):

    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else:
      B,T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)


    return logits, loss


  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, loss = self(idx)
      logits = logits[:,-1,:]
      probs = F.softmax(logits, dim = -1)
      idx_next = torch.multinomial(probs, num_samples = 1)
      idx = torch.cat((idx,idx_next), dim = 1)

    return idx


m = BigramLanguageModel(vocab_size)
logits,loss = m(xb,yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype = torch.long)
print(decode(m.generate(idx,max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.9670, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [18]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [31]:
batch_size = 32
for steps in range(10000):

  xb,yb = get_batch('train')

  logits, loss = m(xb,yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.4411540031433105


In [32]:
idx = torch.zeros((1,1), dtype = torch.long)
print(decode(m.generate(idx,max_new_tokens=500)[0].tolist()))


TH:
MPUSongranewis grot ar' N wol hd!
R:
ARoo papousulo r CEDUTYot!
CIUTy he
ADEneyonqur
MABut PORGLICEThay, pers?
t bance m fof we'sthe head mowak S:
We ntil hily bey be ss awof o lains ooveserowa
Wanthe;

QUMagraim y tod f ir aru ses ban lou sllto le hen rinulindenk

I in w! d e hag encat f hishes
Y aclivild


Kir'l ckind
Can d Awindu, hime, BEDUCI's,
I 'shin.
Ifothor?
l as, ier,
Torenthanot t d ped t alap.

Nouthawir
WIUChe ty'sther? wo cais domone a has ttherike y p urse sondeoun,
'Te by d I


Mathematical Trick in self attention

In [33]:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [37]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [38]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [39]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [43]:
## version 4: self attention
torch.manual_seed(1337)
B,T,C =  4,8,32

x = torch.randn(B,T,C)

head_size = 16
key = nn.Linear(C,head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)
k = key(x)
q = query(x)
wei = q@k.transpose(-2,-1)

tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)
v = value(x)
out = wei @ v

out.shape


torch.Size([4, 8, 16])

In [42]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)